In [1]:
import sys
sys.path.append("../../src/models")

In [2]:
import numpy as np
from actions import city_restrictions, costs

In [3]:
from simulate_pandemic import init_infection, spread_infection, lambda_leak_expose, update_population
from simulate_pandemic import main

In [4]:
from numpy.random import default_rng
import pandas as pd
from datetime import datetime
from tqdm import tqdm

In [5]:
from joblib import Parallel, delayed

In [6]:
def make_df(week, sim, action, data):
    df = pd.DataFrame(pd.Series(data[:, 1]).value_counts()).T
    df['simulation'] = sim
    df['week'] = week
    df['action'] = action
    df = df.rename(columns = {
                              -1 : 'removed',
                               0 : 'susceptible',
                               1 : 'exposed',
                               2 : 'infected',
                               3 : 'hospitalized'
                              }
                  )
    
    return df

In [37]:
def simulate_and_save(policy_number, policy, n_sims, step_size=7, folder='generated_sims/'):
    dfs = []
    for sim in range(1, n_sims+1):
        
        sim_name = f'{policy_number}_{sim}'
                
        data, pop_matrix = main(gpickle_path=gpickle_path,
                                p_r=p_r,
                                policy=policy,
                                disable_tqdm=True,
                                days=step_size*len(policy),
                                step_size=step_size)
        
        return data
        weekly_data = [data[i] for i in range(int(7*len(policy)))]
        df = pd.concat([make_df(week, sim_name, action, data)
                             for week, (data, action) in enumerate(zip(weekly_data, policy))])
        #df.to_parquet(folder + f'{sim_name}.parquet')
        dfs += [df]
    return dfs

In [38]:
rng = default_rng(None)

gpickle_path = "../../data/processed/SP_multiGraph_Job_Edu_Level.gpickle"

prhome = 0.06
p_r = {
    'home'    :  prhome,
    'neighbor':  .1*prhome,
    'work'    :  .1*prhome,
    'school'  :  .15*prhome,
}

In [39]:
costs_keys = list(costs.keys())
costs_values = np.array(list(costs.values()))
p = (1 - costs_values) / (1 - costs_values).sum()
policy = [rng.choice(costs_keys, size=int(364/14), replace=True, p=p) for i in range(6)]
assert len(set([tuple(p) for p in policy])) == len(policy)

In [40]:
res = []
n_sims=1
for i, pol in tqdm(enumerate(policy), total=len(policy)):
    res.append(simulate_and_save(i, pol, n_sims))

  0%|          | 0/6 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [41]:
res = simulate_and_save(i, ['Unrestricted']*52, n_sims, step_size=7)

In [44]:
len(res)

364

In [54]:
ds = [pd.Series(res[i][:, 1]).value_counts() for i in range(1, 364, 7)]

In [56]:
pd.DataFrame(ds)

,-1,0,1,2,3
0,NaN,55458.0,31.0,3.0,NaN
1,8.0,55424.0,24.0,34.0,2.0
2,42.0,55380.0,26.0,39.0,5.0
3,83.0,55304.0,51.0,52.0,2.0
4,140.0,55204.0,69.0,77.0,2.0
5,218.0,55089.0,81.0,98.0,6.0
6,314.0,54962.0,80.0,130.0,6.0
7,458.0,54843.0,78.0,107.0,6.0
8,564.0,54721.0,89.0,111.0,7.0
9,674.0,54606.0,85.0,122.0,5.0


In [51]:
pd.Series(ds[20][:, 1]).value_counts()

 0    53374
-1     1934
 2      101
 1       77
 3        6
dtype: int64

In [33]:
(8/54000)*100

0.014814814814814815

In [20]:
res[0][0]

,susceptible,exposed,simulation,week,action,infected,hospitalized,removed
0,55460,32,0_1,0,Unrestricted,NaN,NaN,NaN
0,55458,29,0_1,1,Social Distancing,5.0,NaN,NaN
0,55452,33,0_1,2,Social Distancing,7.0,NaN,NaN
0,55450,28,0_1,3,Light Quarantine,13.0,1.0,NaN
0,55444,27,0_1,4,Lockdown,19.0,1.0,1.0
0,55440,25,0_1,5,Social Distancing,24.0,1.0,2.0
0,55429,28,0_1,6,Unrestricted,30.0,1.0,4.0
0,55425,28,0_1,7,Hard Quarantine,31.0,1.0,7.0
0,55419,30,0_1,8,Unrestricted,31.0,1.0,11.0
0,55411,30,0_1,9,Light Quarantine,35.0,1.0,15.0


In [11]:
n_sims=4
res = Parallel(n_jobs=-1)(delayed(simulate_and_save)(i, pol, n_sims) 
                                      for i, pol in tqdm(enumerate(policy)))

6it [00:00, 132.98it/s]


IndexError: list index out of range